# Programming for Data Science and Artificial Intelligence

## Supervised Learning - Regression

### Readings: 
- [GERON] Ch4
- [VANDER] Ch5
- [HASTIE] Ch3
- https://scikit-learn.org/stable/modules/linear_model.html

In [25]:
import numpy as np
import matplotlib.pyplot as plt

## Linear Regression

Regression is a supervised algorithm to make prediction based on continous y values.  

For example, given the following data:

| | Egg price  | Gold price    | Oil price   | GDP   |
|---:|:-------------|:-----------|:------|:------|
| 1 | 3  | 100       | 4   | 21   |
| 2 | 4  | 500    | 7   | 43     |

We want to use egg price, gold price and oil price to predict GDP.  We called egg price, gold price, oil price **features**. We called what we want to predict **labels** or **targets**.  Each row is called **sample**.  

#### Course Notations and Terms

*This is important.  Although we are only talking only about linear regression, most of these notations and terms applied to other algorithms as well.*

To make our life easier later, we shall use the following notations throughout our course.  $x_j^{(i)}$ is used to represent the i-th sample, and j-th feature. For example, $x_1^{(1)}$ denote egg price of the first sample (i.e., 100), $x_2^{(1)}$ for gold price of the first sample (i.e., 4), and $x_3^{(1)}$ for oil price of the first sample (i.e., 21).  We use bold captial $X$ to denote the whole matrix of features with $m$ rows of samples and $n$ columns of features.  For compliance to python, we shall called that the **shape** of $X$ as $(m, n)$.  We use $y^{(i)}$ to represent the targets/labels of the i-th sample, where when we do not specify the $i$, i.e., $y$ refers to the whole vector of targets with shape of $(m, )$

We shall define **hypothesis function** as function which given input $X$, will output the predicted $y$.  In machine learning algorithms, we must **learn** and **train** this function to output predicted y as close to actual y.  To differentiate between actual and predicted $y$, we commonly called predicted $y$ as $\hat{y}$ (read as yhat)

For linear regression, the hypothesis function (denoted as $h_{\theta}(x)$ which means $h$ depends on $x$ parametize by $\theta$) is defined as followed.  

\begin{align*}
h_\theta(x) &= \theta_0 + \theta_1x_1 + \theta_2x_2 + \cdots + \theta_nx_n \tag{A}\\
&= \Sigma_{i=0}^n \theta_ix_i  \tag{B}\\
&= \theta^Tx \tag{C}
\end{align*}

Here $\theta$ are called **parameters** or **weights** or **coefficients** that parameterize the linear mappings from $X$ -> $y$.  Also, we commonly don't write equations in the (A) form.  We called (B) form as the summation form and (C) as the **vectorized** form.  Since vectorized form resemble closely to python programming and will make our code clean without many for loops, this course will write in vectorized form when possible.  Note that in equation B, to be mathematically correct, we make $x_0 = 1$ for the intercept term.

We shall further define a **training** set which is subset of the whole dataset used to **train** the hypothesis function so that it can accurately predict $y$.  The resulting hypothesis function is called **model**.  We also shall define a **testing** set which is also a subset of the whole dataset used to **test** the score of the model.  The common practice is to split $70/80\%$ for training set and $30/20\%$ for testing set.

#### Gradient Descent

As you can probably realize, training here refers to training the $\theta$.  How do we pick or learn the paremeters $\theta$?  One reasonable way is to make $h_\theta(x)$ close to $y$, at least for the training examples we have.  To formalize this, we shall define a function that measures, for a given $\theta$, how close the $h(x^{(i)})$ are to the corresponding $y^{(i)}$.  We called this function the **cost function** or **loss function** (denoted as $J$):

$$J(\theta) = \frac{1}{2}\Sigma_{i=1}^m(h_\theta(x^{(i)}) - y^{(i)})^2$$

We want to choose $\theta$ so as to minimize $J(\theta)$.  Such minimization problem will be generally called as **optimization problems**.  Any solution that can help us find the optimal values is called **optimization algorithms**.  Commonly, in such optimization problem, we starts with some random values for $\theta$ (e.g., zeros), and then repeatedly changes $\theta$ to make $J(\theta)$ smaller, until hopefully we converge to a value of $\theta$ that minimizes $J(\theta)$.  

One such optimization algorithm to do that is called **gradient descent** algorithm, which starts with some initial $\theta$, and repeatedly performs the update:

$$\theta_j := \theta_j - \alpha * \frac{\partial J}{\partial \theta_j}$$
 
This update is simultaneously performed for all values of $j=0, 1, \cdots, n$.  Here, $\alpha$ is called the **learning rate**, ranging from 0 to 1.  Commonly, we tried 0.001 as default, and this value must be **fine-tuned** in order to know what value is the best.  Any parameters such as learning rate that we need to fine-tuned through trial and errors are called **hyperparameters**.

In order to implement this algorithm, we have to find the **partial derivative of the loss function in respect to each $\theta_j$**.  Let's try the partial derivative of our loss function in respect to $\theta_j$.  Also, let's first work it out for only one training example first as follows:

$$
\begin{aligned}
\frac{\partial J}{\partial \theta_j} &= \frac{\partial}{\partial \theta_j} \frac{1}{2}(h_\theta(x) - y)^2 \\
&= 2 * \frac{1}{2} (h_\theta(x) - y) * \frac{\partial}{\partial \theta_j} (h_\theta(x) - y) \\
&= 2 * \frac{1}{2} (h_\theta(x) - y) * \frac{\partial}{\partial \theta_j} \big(\Sigma_{i=0}^n \theta_ix_i - y\big) \\
&= (h_\theta(x) - y)x_j
\end{aligned}
$$

This rule has several properties that seem natural and intuitive. For instance, the magnitude of the update is proportional to the **error** term $h_\theta(x) - y$; thus, for instance, if we are encountering a training example on which our prediction nearly matches the actual value of $y^{(i)}$, then we find that there is little need to change the parameters; in contrast, a larger change to the parameters will be made if our prediction has a large error (i.e., if it is very far from $y^{(i)}$).

To modify the update rule for whole training example, we revise the update rule to include the summation as

$$\theta_j := \theta_j - \alpha * \Sigma_{i=1}^m(h_\theta(x^{(i)}) - y^{(i)})x_j \tag{for every $j$}$$

Notice that minus sign which indicates **gradient descent**.  We can simply flip the sign to plus and called it as **gradient ascent**, both of which give the same answer.

$$\theta_j := \theta_j + \alpha * \Sigma_{i=1}^m(y^{(i)} - h_\theta(x^{(i)}))x_j^{(i)} \tag{for every $j$}$$

Since this gradient descent calculates gradient using every example in the entire training set, we called this as **batch gradient descent**.

Sometimes, performing batch gradient descent can be slow, thus we can use **stochastic gradient descent** which refers to looking at only one training example, where we can pick with or without replacement.  Here, **without replacement** refers to the process in which no same sample is used in the same **epoch**.  Here epoch means one  iteration which the whole training set is being exhausted.  Thus, in without replacement, we simply loop from $i =1$ to $m$ for one epoch.

$$ \theta_j := \theta_j - \alpha *(h_\theta(x)^{(i)}-y^{(i)})x_j \tag{for every $j$}$$

Although **stochastic gradient descent** may be faster, it rarely converges to the optimum given its randomness.  A middle ground is **mini-batch gradient descent** which can be expressed as

$$\theta_j := \theta_j + \alpha * \Sigma_{i=start}^{batchsize}(y^{(i)} - h_\theta(x^{(i)}))x_j^{(i)} \tag{for every $j$}$$

Similarly, we can do this with or without replacement.  In without replacement, we simply chop evenly and exhaust the whole training set for one epoch.

#### Closed form

Gradient descent gives one possible mean for minimizing $J$, which uses iterative approach and may take time.  In the situation where we know that our cost function is strictly concave or convex, we can explicitly take its derivative to zero.  This process of derivation is called obtaining the **normal equations** or **closed form**. 

The **closed form** of linear regression can be derived easily.  Let $X$ be a matrix of shape $(m, n)$, $\theta$ as shape $(n, )$, and $Y$ as vector of shape $(m, )$.  Instead of writing the cost function as power of square, we shall write it in matrix multiplication as follows:

$$\frac{\partial J}{\partial \theta} (X\theta - Y)^T*(X\theta-Y)$$

Recall the following properties:

$$\frac{\partial J}{\partial X} X^TX=2X \tag{A}$$
$$\frac{\partial J}{\partial X} AX=A^T$$
$$(XY)^T = Y^TX^T$$

Therefore

\begin{align*}
\frac{\partial J}{\partial \theta} (X\theta - Y)^T*(X\theta-Y) &= \frac{\partial J}{\partial \theta} (\theta^TX^TX\theta - \theta^TX^TY - Y^TX\theta + Y^TY)\\
&= 2X^TX\theta - 2X^TY \tag{see note*}\\
&= X^TX\theta - X^TY = 0 \tag{set derivative to 0}\\
&= \theta = (X^TX)^{-1}X^TY
\end{align*}


Note*: Since $X\theta$ is a vector, and so is $Y$, it doesn't matter what the order is, thus we can simply add them to 2.  Also, we got 2 in front of the first part because we have two $\theta$ (used the property A)


**Why not closed form always**.  The answer is simple.  It does not always exists or possible, for example, the cost function is not convex or concave.  But of course, if it exists, we usually prefer closed form given that it is usually faster than gradient descent.  Nevertheless, as you can see, taking inverse of huge number of features can be expensive, thus it is also not always straightforward thing to always prefer closed form.

Yes, that's it for most of the theoretical stuff.  Let's start implementing some of these concepts so we can better understand them.

After we implement them from scratch, we shall explore them further using sklearn which will save us a lot of time from implementing from scratch and also comes with a lot of nice utilities.

### Algorithm 1: Closed Form

The closed form is a normal equations derived from setting the derivatives = 0.  By performing only some inverse operations and matrix multiplication, we will be able to get the theta.

$$\theta = (X^TX)^{-1}X^TY$$

When closed form is available, is doable (can be inversed - can use pseudoinverse), and with not many features (i.e., inverse can be slow), it is recommended to always use closed form.  

**Implementation steps:**

1. Prepare your data
    - add intercept
    - $X$ and $y$ and $w$ in the right shape
        - $X$ -> $(m, n)$
        - $y$ -> $(m, )$
        - $w$ -> $(n, )$
        - where $m$ is number of samples
        - where $n$ is number of features
    - train-test split
    - feature scale
    - clean out any missing data
    - (optional) feature engineering
2. Plug everything into the equation.  Here we shall use X_train to retrieve the $\theta$
$$\theta = (X^TX)^{-1}X^TY$$

3. We simply using the $\theta$, we can perform a dot product with our X_test which will give us $\hat{y}$.

4. We then calculate the errors using mean-squared-error function:

$$\frac{1}{m}\Sigma_{i=1}^m(h_\theta(x^{(i)}) - y^{(i)})^2$$

Note that it's a bit different from our $J(\theta)$ because $J(\theta)$ puts $\frac{1}{2}$ instead of $\frac{1}{m}$ for mathematical convenience for derivatives, since we know changing constants do not change the optimization results.


#### 1.1 Get your X and y in the right shape

In [26]:
# 1. Let's load some boston data 
# as our regression case study
from sklearn.datasets import load_boston
# type - Bunch
# Bunch - dictionary of numpy data
# boston.feature_names
# print(boston)
boston = load_boston()

In [27]:
X = boston.data
X.shape #number of samples, number of features

m = X.shape[0]  #number of samples
n = X.shape[1]  #number of features

In [28]:
y = boston.target

In [29]:
# number of rows in X is the same as number of rows in y
# because so we have yhat for all y
assert m == y.shape[0] 

#### 1.2 Feature scale your data to reach faster convergence

In [30]:
# I want to standardize my data so that mean is 0, variance is 1
# average across each feature, NOT across each sample
# Why we need to standardize
# Because standardizing usually allows us to reach convergence faster
# Why -> because the values are within smaller range
# Thus, the gradients are also within limited range, and NOT go crazy

from sklearn.preprocessing import StandardScaler

# 1. StandardScaler.fit(X)  #this scaler (or self) knows the mean and std so now
# it knows how to transform data
# 2  X = StandardScaler.transform(X)  #not in place; will return something

# 1. StandardScaler.fit_transform(X) -> 1 and 2 sequentially

# create an object of StandardScaler
# StandardScaler is a class
# scaler is called instance/object

# ALMOST always, feature scale your data using normalization or standardization
# If you assume your data is gaussian, use standardization, otherwise, you do the normalization

scaler = StandardScaler()

X = scaler.fit_transform(X)

#### 1.3 Train test split your data

In [31]:
# what is the appropriate size for test data
# 70/30 (small dataset); 80/20 (medium dataset); 90/10 (large dataset);
# why large dataset, can set test size to 10, because
# 10% of large dataset is already enough for testing accuracy
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

assert len(X_train)  == len(y_train)
assert len(X_test) == len(y_test)

#### 1.4 Add intercepts

In [32]:
# What is the shape of X they want
# (number of samples, number of features) --> correct shape
# for closed form formula
# How about the intercept
# w0 is OUR intercept
# what is the shape of w -->(n+1, )
# What is the shape of intercept --->(m, 1)
#X = [1 2 3     @  [w0
#     1 4 6         w1
#     1 9 1         w2 
#     1 10 2 ] 

# np.ones((shape))
intercept = np.ones((X_train.shape[0], 1))

# concatenate the intercept based on axis=1
X_train = np.concatenate((intercept, X_train), axis=1)

# np.ones((shape))
intercept = np.ones((X_test.shape[0], 1))

# concatenate the intercept based on axis=1
X_test = np.concatenate((intercept, X_test), axis=1)

#### 1.5. Feature Engineering (optional)

It is sometimes useful to engineer new features (e.g., polynomial, kernels) so to create some non-linear relationships with your target.

Here we gonna skip

### Step 2: Fit your algorithm 

#### 1. Define your algorithm

In [33]:
from numpy.linalg import inv

# order of operation DOES NOT MATTER
# But don't flip y before X^T for example
def closed_form(X, y):
    return inv(X.T @ X) @ X.T @ y

In [34]:
# let's use the closed_form to find the theta
theta = closed_form(X_train, y_train)
theta  #<------this is our model

array([22.36768371, -0.7334575 ,  0.47556167,  0.23019729,  0.73870066,
       -1.557821  ,  3.29499542, -0.25403808, -2.25993984,  2.32837865,
       -2.16534738, -2.04283595,  0.66692148, -3.42962899])

#### 2. Compute accuracy/loss

In [35]:
# Compute the accuracy/loss

yhat = X_test @ theta #==> X (m, n+1)  @ (n+1, ) w ==> (m, ) y

# if I want to compare yhat and y, I need to make sure they are the same shape
assert y_test.shape == yhat.shape

In [36]:
# get the mse
mse = ((y_test - yhat)**2).sum() / X_test.shape[0]
print("Mean squared errors: ", mse)

Mean squared errors:  26.451245174252406


### Algorithm 2: Batch Gradient Descent

The gradient descent has the following steps:

1. Prepare your data
    - add intercept
    - $X$ and $y$ and $w$ in the right shape
        - $X$ -> $(m, n)$
        - $y$ -> $(m, )$
        - $w$ -> $(n, )$
    - train-test split
    - feature scale
    - clean out any missing data
    - (optional) feature engineering
2. Predict and calculate the loss
    - The loss function is the mean squared error
    $$J = \frac{\Sigma_{i=1}^{m}(h-y)^2}{2}$$
    where $h$ is simply
    $$ h = \theta^Tx $$
3. Calculate the gradient based on the loss
    - The gradient of the loss function is
    $$\frac{\partial J}{\partial \theta_j} = \Sigma_{i=1}^{m}(h^{(i)}-y^{(i)})x_j$$
4. Update the theta with this update rule
    $$\theta_j := \theta_j - \alpha * \frac{\partial J}{\partial \theta_j}$$
    where $\alpha$ is a typical learning rate range between 0 and 1
5. Loop 2-4 until max_iter is reached, or the difference between old loss and new loss are smaller than some predefined threshold tol

#### 1. Define your algorithm

In [117]:
from time import time

# Step 1: Prepare your data
# X_train, X_test have intercepts that are being concatenated to the data
# [1, features
#  1, features....]

# making sure our X_train has same sample size as y_train
assert X_train.shape[0] == y_train.shape[0]

# initialize our w
# We don't have to do X.shape[1] + 1 because our X_train already has the
# intercept
# w = theta/beta/coefficients
theta = np.zeros(X_train.shape[1])

# define the learning rate
# later on, you gonna know that it should be better to make it slowly decreasing
# once we perform a lot of iterations, we want the update to slow down, so it converges better
alpha = 0.0001

# define our max_iter
# typical to call it epochs <---ml people likes to call it
max_iter = 1000

loss_old = 10000

tol = 0.0001

iter_stop = 0

def h_theta(X, theta):
    return X @ theta

def mse(yhat, y):
    ans = (((yhat - y)**2).sum()) / yhat.shape[0]
    return ans

def gradient(X, error):
    return X.T @ error

start = time()

# define your for loop
for i in range(max_iter):
    
    # 1. yhat = X @ w
    # prediction
    # yhat (m, ) = (m, n) @ (n, )
    yhat = h_theta(X_train, theta)

    # 2. error = yhat - y_train
    # error for use to calculate gradients
    # error (m, ) = (m, ) - (m, )
    error = yhat - y_train

    # 3. grad = X.T @ error
    # grad (n, ) = (n, m) @ (m, )
    # grad for each feature j
    grad = gradient(X_train, error)

    # 4. w = w - alpha * grad
    # update w
    # w (n, ) = (n, ) - scalar * (n, )
    theta = theta - alpha * grad

time_taken = time() - start

#### 2. Compute accuracy/loss

In [118]:
# we got our lovely w
# now it's time to check our accuracy
# 1. Make prediction
yhat = h_theta(X_test, theta)

# 2. Calculate mean squared errors
mse_result = mse(yhat, y_test)
# print the mse
print("MSE: ", mse_result)
print("Stop at iteration: ", iter_stop)
print("Time used: ", time_taken)

MSE:  26.525588464427056
Stop at iteration:  0
Time used:  0.007861852645874023


### Algorithm 3: Stochastic Gradient Descent

The gradient descent has the following steps:

1. Prepare your data
    - add intercept
    - $X$ and $y$ and $w$ in the right shape
        - $X$ -> $(m, n)$
        - $y$ -> $(m, )$
        - $w$ -> $(n, )$
    - train-test split
    - feature scale
    - clean out any missing data
    - (optional) feature engineering
2. Predict and calculate the loss
3. Calculate the gradient based on the loss
    - **This differs from batch gradient descent that it only uses one sample to estimate the loss and gradient**
        $$\frac{\partial J}{\partial \theta_j} = (h^{(i)}-y^{(i)})x_j$$
4. Update the theta
5. Loop 2-4 until max_iter is reached, or the difference between old loss and new loss are smaller than some predefined threshold tol

### Algorithm 4: Mini-Batch Gradient Descent

The gradient descent has the following steps:

1. Prepare your data
    - add intercept
    - $X$ and $y$ and $w$ in the right shape
        - $X$ -> $(m, n)$
        - $y$ -> $(m, )$
        - $w$ -> $(n, )$
    - train-test split
    - feature scale
    - clean out any missing data
    - (optional) feature engineering
2. Predict and calculate the loss
3. Calculate the gradient based on the loss
    - **This differs from batch gradient descent that it only uses a subset of samples to estimate the loss and gradient**
        $$\frac{\partial J}{\partial \theta_j} = \Sigma_{i=start}^{batch}(h^{(i)}-y^{(i)})x_j$$
    where start is a randomized number within the range of $m$ and batch is a predefined batch size, typically around 100 to 500
4. Update the theta
5. Loop 2-4 until max_iter is reached, or the difference between old loss and new loss are smaller than some predefined threshold tol

I will not implement this, but leave to your exercise.  Enjoy!

### === Task ===

1. Implement early stopping in which if the absolute difference between old loss and new loss does not exceed certain threshold, we abort the learning.

2. Implement options for stochastic gradient descent in which we use only one sample for training.  Make sure that sample does not repeat unless all samples are read at least once already.

3. Add options for mini-batch gradient descent.

3. Put everything into class.

In [120]:
max_iter = 1000

loss_old = 10000

tol = 0.00001

iter_stop = 0

start = time()

# define your for loop
for i in range(max_iter):
    
    # 1. yhat = X @ w
    # prediction
    # yhat (m, ) = (m, n) @ (n, )
    yhat = h_theta(X_train, theta)

    # 2. error = yhat - y_train
    # error for use to calculate gradients
    # error (m, ) = (m, ) - (m, )
    error = yhat - y_train
    
    # 3. grad = X.T @ error
    # grad (n, ) = (n, m) @ (m, )
    # grad for each feature j
    grad = gradient(X_train, error)

    # 4. w = w - alpha * grad
    # update w
    # w (n, ) = (n, ) - scalar * (n, )
    theta = theta - alpha * grad

    loss_new = mse(yhat, y_train)
    diff = np.abs(loss_new - loss_old)
    #print(i, diff)

    if diff < tol:
        iter_stop = i+1
        break
    else:
        loss_old = loss_new
        


time_taken = time() - start
print("Stop at iteration: ", iter_stop)
print("Time used: ", time_taken)

Stop at iteration:  2
Time used:  0.0
